In [ ]:
import pandas as pd
import concurrent.futures
from functools import partial
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import time
import pandas as pd
import concurrent.futures
from functools import partial
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
import time
from tqdm import tqdm


import os
def create_session():
    session = requests.Session()

    retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
    headers = {
        'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 16_5_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.5 Mobile/15E148 Safari/604.1',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1'
    }
    session.headers=headers
    session.mount('https://', HTTPAdapter(max_retries=retries))
    return session

def playerpull(session, player_id, team_id, season, opp=False, ps=False):

    term = "Opponent" if opp else "Team"
    s_type = "Playoffs" if ps else "Regular Season"
    
    wowy_url = "https://api.pbpstats.com/get-wowy-stats/nba"
    
    params_on = {
        "0Exactly1OnFloor": player_id,
        "TeamId": team_id,
        "Season": season,
        "SeasonType": s_type,
        "Type": term
    }
   
    
    params_off = {
        "0Exactly0OnFloor": player_id,
        "TeamId": team_id,
        "Season": season,
        "SeasonType": s_type,
        "Type": term
    }
    
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.183',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'DNT': '1',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Sec-Fetch-User': '?1',
        'Cache-Control': 'max-age=0',}

        response_on = session.get(wowy_url, params=params_on, timeout=60)
        response_on.raise_for_status()
        player_stats_on = response_on.json()["single_row_table_data"]
        time.sleep(.5)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36',}
        response_off = session.get(wowy_url, params=params_off, timeout=60)
        response_off.raise_for_status()
        player_stats_off = response_off.json()["single_row_table_data"]
        print('Row')
        time.sleep(.5)
        
        # Instead of creating dataframes and concatenating, we'll return the raw data
        return player_stats_on, player_stats_off
    except (requests.exceptions.RequestException, KeyError) as e:
        print(f"Error processing player {player_id}: {str(e)}")
        return None, None

def process_player(session, player_id, team_id, year, opp, ps):
    try:
        season = f"{year-1}-{str(year)[-2:]}"
        stats_on, stats_off = playerpull(session, player_id, team_id, season, opp=opp, ps=ps)
        if stats_on is not None and stats_off is not None:
            # Add the additional fields to both stat dictionaries
            for stats in [stats_on, stats_off]:
                stats['player_id'] = player_id
                stats['team_id'] = team_id
                stats['year'] = year
                stats['season'] = season
                stats['player_vs'] = opp
            
            stats_on['player_on'] = True
            stats_off['player_on'] = False
            
            # Return a list of two dictionaries instead of a dataframe
            return [stats_on, stats_off]
        return None
    except Exception as e:
        print(f"Error processing player {player_id}: {str(e)}")
        return None

def pull_onoff(years, opp=False, ps=False):
    # ... (previous code remains the same)
    opp_string = 'vs' if opp else ''
    pstring = 'ps' if ps else ''
    
    player_index = pd.read_csv('https://raw.githubusercontent.com/gabriel1200/shot_data/master/player_index'+pstring+'.csv')
    player_index = player_index[player_index.year > 2000]
    
    session = create_session()
    
    for year in years:
        season_index = player_index[player_index.year == year].reset_index(drop=True)
        old_file = (str(year))+opp_string+pstring+'.csv'
        print(old_file)
        if os.path.exists(old_file):
            old_df = pd.read_csv(old_file)
            old_df['total_id'] = old_df['player_id']+old_df['team_id']
            season_index['total_id']  = season_index['PLAYER_ID']+season_index['TEAM_ID']
            total_ids = old_df['total_id'].tolist()
            season_index = season_index[~season_index.total_id.isin(total_ids)]
            season_index.drop(columns='total_id',inplace=True)
        print(f"Processing {len(season_index)} players for year {year}")
        
        process_func = partial(process_player, session, year=year, opp=opp, ps=ps)
        # ... (previous code remains the same)
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures = [executor.submit(process_func, row['PLAYER_ID'], row['TEAM_ID']) 
                       for _, row in season_index.iterrows()]
            
            all_stats = []
            for future in concurrent.futures.as_completed(futures):
                result = future.result()
                if result is not None:
                    all_stats.extend(result)
        
        if all_stats:
            # Create a dataframe from all collected stats at once
            year_frame = pd.DataFrame(all_stats)
            if os.path.exists(old_file):
                old_df =pd.read_csv(old_file)
                year_frame = pd.concat([old_df,year_frame])
            print(f'Year {year} completed, saving...')
            year_frame.to_csv(f"{year}{opp_string}{pstring}.csv", index=False)
        else:
            print(f"No data to save for year {year}")


if __name__ == "__main__":
    years = list(range(2001, 2025))
    pull_onoff(years, opp=False, ps=True)